# Working with the B3DB brain-blood barrier dataset
> 

- toc: false 
- badges: true
- comments: true
- categories: [python, bioinformatics, datasets, SMILES, cheminformatics, datamol, RDKit, molecules]
- image: images/bloodcell.gif